![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

### [Docker, Spark, and Iceberg: The Fastest Way to Try Iceberg!](https://tabular.io/blog/docker-spark-and-iceberg/)

In [ ]:
!pip install pyspark-opendic==0.1.11

In [ ]:
from pyspark.sql import SparkSession

def read_secret(secret_name):
    """ Get `secret_name` from docker-compose secret store"""
    secret_path = f"/run/secrets/{secret_name}"
    try:
        with open(secret_path, "r") as f:
            return f.read().strip()  # Remove any trailing newline
    except FileNotFoundError:
        print(f"Secret {secret_name} not found.")
        return None


## DEFINE SENSITIVE VARIABLES
POLARIS_CATALOG_NAME = 'polaris'
ENGINEER_CLIENT_ID = read_secret("engineer_client_id")
ENGINEER_CLIENT_SECRET = read_secret("engineer_client_secret")
ADLS_IO="org.apache.iceberg.azure.adlsv2.ADLSFileIO"
FILE_IO="org.apache.iceberg.io.ResolvingFileIO"

def create_session(client_id, client_secret, scope, fileio_impl):
    spark = (SparkSession.builder
        .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.0,software.amazon.awssdk:bundle:2.28.17,software.amazon.awssdk:url-connection-client:2.28.17")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
        .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.polaris.type", "rest")
        .config("spark.sql.catalog.polaris.warehouse", POLARIS_CATALOG_NAME)
        .config("spark.sql.catalog.polaris.uri", 'http://polaris:8181/api/catalog')
        .config("spark.sql.catalog.polaris.credential", f"{client_id}:{client_secret}")
        .config("spark.sql.catalog.polaris.scope", 'PRINCIPAL_ROLE:ALL')
        .config("spark.sql.defaultCatalog", "polaris")
        .config("spark.sql.catalogImplementation", "in-memory")
        .config("spark.driver.extraJavaOptions", "-Divy.cache.dir=/tmp -Divy.home=/tmp")
        .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")
        .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation", 'vended-credentials')
        .config("spark.sql.catalog.polaris.io-impl", fileio_impl)
        .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")).getOrCreate()
        
    print("Spark Running")
    return spark


## Start Spark Session
spark = create_session(client_id=ENGINEER_CLIENT_ID, client_secret=ENGINEER_CLIENT_SECRET, scope='PRINCIPAL_ROLE:ALL',fileio_impl=FILE_IO )
spark

## Setting up pyspark-opendic

In [ ]:
from pyspark_opendic.catalog import OpenDicCatalog

# Init polarisx catalog
POLARIS_URI= "http://polaris:8181/api"
catalog = OpenDicCatalog(spark, POLARIS_URI)
print("Catalog initialized")

In [ ]:
catalog.sql("Show namespaces").show()

In [ ]:
%%sql
use SYSTEM

### Define the schema for a andfunc object

In [ ]:
catalog.sql(
    """
    DEFINE OPEN andfunc
    props {
        "args": "MAP",
        "language": "STRING",
        "def": "string",
        "runtime": "string"
    }
    """
)

### Create a new andfunc

In [ ]:
catalog.sql(
 """CREATE OPEN andfunc foo 
    props {
        "args": {
            "arg1": "string",
            "arg2": "number"
        },
        "language": "python",
        "def": "def foo(arg1, arg2):    return arg1 + arg2",
        "runtime": "3.13"
    }
    """
)

### Create Mapping to snowflake

### List objects

In [ ]:
catalog.sql(
    """
    SHOW OPEN TYPES
    """
)

In [ ]:
catalog.sql(
    """
    SHOW OPEN andfunc
    """
)

### Drop andfunc

In [ ]:
catalog.sql(
    """
    DROP OPEN andfunc
    """
)


### Visualize opendic tables

In [ ]:
%%sql
show tables in SYSTEM

In [ ]:
%%sql
select * from andfunc

In [ ]:
%%sql
DESCRIBE EXTENDED SYSTEM.andfunc

## Load One Month of NYC Taxi/Limousine Trip Data

For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.

To be able to rerun the notebook several times, let's drop the table if it exists to start fresh.

In [ ]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc

In [ ]:
%%sql

DROP TABLE IF EXISTS nyc.taxis

In [ ]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.write.saveAsTable("nyc.taxis")

In [ ]:
%%sql

DESCRIBE EXTENDED nyc.taxis

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

## Schema Evolution

Adding, dropping, renaming, or altering columns is easy and safe in Iceberg. In this example, we'll rename `fare_amount` to `fare` and `trip_distance` to `distance`. We'll also add a float column `fare_per_distance_unit` immediately after `distance`.

In [ ]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN fare_amount TO fare

In [ ]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN trip_distance TO distance

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance COMMENT 'The elapsed trip distance in miles reported by the taximeter.'

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance TYPE double;

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance AFTER fare;

In [ ]:
%%sql

ALTER TABLE nyc.taxis
ADD COLUMN fare_per_distance_unit float AFTER distance

Let's update the new `fare_per_distance_unit` to equal `fare` divided by `distance`.

In [ ]:
%%sql

UPDATE nyc.taxis
SET fare_per_distance_unit = fare/distance

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

## Expressive SQL for Row Level Changes
With Iceberg tables, `DELETE` queries can be used to perform row-level deletes. This is as simple as providing the table name and a `WHERE` predicate. If the filter matches an entire partition of the table, Iceberg will intelligently perform a metadata-only operation where it simply deletes the metadata for that partition.

Let's perform a row-level delete for all rows that have a `fare_per_distance_unit` greater than 4 or a `distance` greater than 2. This should leave us with relatively short trips that have a relatively high fare per distance traveled.

In [ ]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit > 4.0 OR distance > 2.0

There are some fares that have a `null` for `fare_per_distance_unit` due to the distance being `0`. Let's remove those as well.

In [ ]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit is null

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

## Partitioning

A table’s partitioning can be updated in place and applied only to newly written data. Query plans are then split, using the old partition scheme for data written before the partition scheme was changed, and using the new partition scheme for data written after. People querying the table don’t even have to be aware of this split. Simple predicates in WHERE clauses are automatically converted to partition filters that prune out files with no matches. This is what’s referred to in Iceberg as *Hidden Partitioning*.

In [ ]:
%%sql

ALTER TABLE nyc.taxis
ADD PARTITION FIELD VendorID

## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [ ]:
%%sql

SELECT snapshot_id, manifest_list
FROM nyc.taxis.snapshots

The `files` table contains loads of information on data files, including column level statistics such as null counts, lower bounds, and upper bounds.

In [ ]:
%%sql

SELECT file_path, file_format, record_count, null_value_counts, lower_bounds, upper_bounds
FROM nyc.taxis.files

## Time Travel

The history table lists all snapshots and which parent snapshot they derive from. The `is_current_ancestor` flag let's you know if a snapshot is part of the linear history of the current snapshot of the table.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

You can time-travel by altering the `current-snapshot-id` property of the table to reference any snapshot in the table's history. Let's revert the table to it's original state by traveling to the very first snapshot ID.

In [ ]:
%%sql --var df

SELECT *
FROM nyc.taxis.history

In [ ]:
original_snapshot = df.head().snapshot_id
spark.sql(f"CALL system.rollback_to_snapshot('nyc.taxis', {original_snapshot})")
original_snapshot

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

Another look at the history table shows that the original state of the table has been added as a new entry
with the original snapshot ID.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis